In [12]:
# import preprocess as prep
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 
import importlib
import preprocess_lite
import preprocess
import SLR_model
from scipy. stats import mode
import tensorflow as tf
import numpy as np
# model will output multiple(15) results/sec, how are we gonna handle it?

In [13]:
importlib.reload(SLR_model)
importlib.reload(preprocess_lite)
importlib.reload(preprocess)



<module 'preprocess' from 'f:\\home\\shd\\BomNae-SLR\\preprocess.py'>

In [14]:
# Configurations
model_dir = "saves"
THRESHOLD = 0.5
# batch_size = 16
file_name="11-20-16-30epochs-5times_final"


In [15]:
# 세이브된 모델 로드
load_path = os.path.join(model_dir, "ckpt",file_name+".keras")
model = SLR_model.load_model(load_path)


In [22]:
# load data
l_raws=[]
r_raws=[]
p_raws=[]
y_raws=[]
person_id = 18
# TODO hardcode path
# a video
for j in sorted(os.listdir(os.path.join(preprocess_lite.getoutputdir("val"),str(person_id)))):
    j="0006.npz"
    l_raw, r_raw, p_raw, y_raw= preprocess_lite.load_data(f"{person_id}/{j}","val")
    
    l_raws.append(l_raw)
    r_raws.append(r_raw)
    p_raws.append(p_raw)
    y_raws.append(y_raw)
    l_test, each = SLR_model.serialize(l_raws)
    r_test, each = SLR_model.serialize(r_raws)
    p_test, each = SLR_model.serialize(p_raws, stride=2,is_pose = True)
    x_test = (l_test, r_test, p_test)

    predictions = model.predict(x_test,32)
    pred=SLR_model.decode_onehot2d(predictions)
    for i in range(len(predictions)):
        print(f"{y_raw}:{pred[i]}")
        if pred[i]==y_raw:
            print(tf.math.top_k(predictions[i],k=2).values.numpy())
            break
            
    # print(SLR_model.decode_onehot2d(predictions))

    break
    

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
1232:811
1232:811
1232:811
1232:811
1232:811
1232:811
1232:811
1232:467
1232:347
1232:1972
1232:1908
1232:1232
[0.80256814 0.11912374]


In [23]:
# load data
l_raws=[]
r_raws=[]
p_raws=[]
y_raws=[]
person_id = 1

name_list=[]
# TODO hardcode path
# a video
for person_id in range(17,18):
    count=0
    incorrect=0
    if person_id==2:
        continue
    # val_list=sorted(os.listdir(os.path.join(preprocess_lite.getoutputdir(),str(person_id))))
    val_list=sorted(os.listdir(os.path.join(preprocess_lite.getoutputdir('val'),str(person_id))))
    
    for j in val_list:
        # j="0001.npz"
        try:
            l_raw, r_raw, p_raw, y_raw = preprocess_lite.load_data(f"{person_id}/{j}",'val')
            # l_raw, r_raw, p_raw, y_raw,w = preprocess_lite.load_data(f"{person_id}/{j}")
            
        except Exception as e:
            print(e)
            continue
        l_raws.append(l_raw)
        r_raws.append(r_raw)
        p_raws.append(p_raw)
        y_raws.append(y_raw)
        l_test, each = SLR_model.serialize(l_raws)
        r_test, each = SLR_model.serialize(r_raws)
        p_test, each = SLR_model.serialize(p_raws, stride=2,is_pose = True)
        x_test = (l_test, r_test, p_test)
        try:
            predictions = model.predict(x_test,32,verbose=0)
            pred=SLR_model.decode_onehot2d(predictions)
            num=mode(pred)
            for i in range(len(predictions)):
                if pred[i]==y_raw:
                    top=tf.math.top_k(predictions[i],k=2).values.numpy()
                    count+=1
                    break
                else: top=np.array([0,0]) 
            ans=(abs(top[0]-top[1]))
        except Exception as e:
            log_msg=f'{j}) {e}\n'
        else:
            log_msg=f'{j}):correct:{y_raw}/{num[0]}\n'
    
            if 0==ans:
                incorrect+=1
                # name_list.append(j)
        finally:
            # with open(file_name+"_test_log.txt","a") as log:
                # if int(y_raw)!=int(num[0]):
                # log.write(log_msg)
                # print(log_msg)
            l_raws.clear()
            r_raws.clear()
            p_raws.clear()
            y_raws.clear()
    print(person_id)
    print(count/len(val_list)*100)
    print()

c:\Users\shd\anaconda3\envs\BomNae-SLR\lib\contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


17
70.36144578313252



In [ ]:
# load data
l_raws=[]
r_raws=[]
p_raws=[]
y_raws=[]
person_id = 1

name_list=[]
# TODO hardcode path
# a video
for person_id in range(17,19):
    count=0
    incorrect=0
    if person_id==2:
        continue
    # val_list=sorted(os.listdir(os.path.join(preprocess_lite.getoutputdir(),str(person_id))))
    val_list=sorted(os.listdir(os.path.join(preprocess.getoutputdir('val'),str(person_id))))
    
    for j in val_list:
        # j="0001.npz"
        try:
            l_raw, r_raw, p_raw, y_raw = preprocess.load_data(f"{person_id}/{j}",'val')
            # l_raw, r_raw, p_raw, y_raw,w = preprocess_lite.load_data(f"{person_id}/{j}")
            
        except Exception as e:
            print(e)
            continue
        l_raws.append(l_raw)
        r_raws.append(r_raw)
        p_raws.append(p_raw)
        y_raws.append(y_raw)
        l_test, each = SLR_model.serialize(l_raws)
        r_test, each = SLR_model.serialize(r_raws)
        p_test, each = SLR_model.serialize(p_raws, stride=2,is_pose = True)
        x_test = (l_test, r_test, p_test)
        try:
            predictions = model2.predict(x_test,32,verbose=0)
            pred=SLR_model.decode_onehot2d(predictions)
            num=mode(pred)
            for i in range(len(predictions)):
                if pred[i]==y_raw:
                    top=tf.math.top_k(predictions[i],k=2).values.numpy()
                    count+=1
                    break
                else: top=np.array([0,0]) 
            ans=(abs(top[0]-top[1]))
        except Exception as e:
            log_msg=f'{j}) {e}\n'
        else:
            log_msg=f'{j}):correct:{y_raw}/{num[0]}\n'
    
            if 0==ans:
                incorrect+=1
                # name_list.append(j)
        finally:
            # with open(file_name+"_test_log.txt","a") as log:
                # if int(y_raw)!=int(num[0]):
                # log.write(log_msg)
                # print(log_msg)
            l_raws.clear()
            r_raws.clear()
            p_raws.clear()
            y_raws.clear()
    print(person_id)
    print(count/len(val_list)*100)
    print()

In [ ]:
# # dummy x_raw data
# l_raws = []
# duration_list = []
# h_elem = np.zeros(shape=(4,5,3))
# # p_elem = np.zeros(shape=(3,10))
# # comb_elem = [h_elem, h_elem, p_elem]
# x_elem = []
# duration = (np.random.randint(90,100))
# duration_list.append(duration)
# for j in range(duration): # frame size
#     x_elem.append(h_elem)
#     # x_elem.append(comb_elem)
# l_raws.append(x_elem)
# r_raws = []
# 
# h_elem = np.zeros(shape=(4,5,3))
# # p_elem = np.zeros(shape=(3,10))
# # comb_elem = [h_elem, h_elem, p_elem]
# x_elem = []
# duration = duration_list[0]
# for j in range(duration): # frame size
#     x_elem.append(h_elem)
#     # x_elem.append(comb_elem)
# r_raws.append(x_elem)
# p_raws = []
# 
# # h_elem = np.zeros(shape=(4,5,3))
# p_elem = np.zeros(shape=(3,10))
# # comb_elem = [h_elem, h_elem, p_elem]
# x_elem = []
# duration = duration_list[0]
# for j in range(duration): # frame size
#     # x_elem.append(h_elem)
#     x_elem.append(p_elem)
#     # x_elem.append(comb_elem)
# p_raws.append(x_elem)

In [ ]:
l_test, each = SLR_model.serialize(l_raws)
r_test, each = SLR_model.serialize(r_raws)
p_test, each = SLR_model.serialize(p_raws, stride=2,is_pose = True)
x_test = (l_test, r_test, p_test)

# dataset = SLR_model.convert_to_dataset_single(x_test, batch_size)


# with open(hist_path, 'w') as file:
#     json.dump(hist.history, file)

In [ ]:
y_raw

In [ ]:

predictions = model.predict(x_test,32)
print(SLR_model.decode_onehot2d(predictions))


In [ ]:
predictions

In [ ]:
# pred_bin = predictions.round().astype(np.int32)
# pred_bin

In [ ]:
importlib.reload(SLR_model)

In [ ]:
SLR_model.decode_onehot2d(predictions)

In [ ]:
SLR_model.bin2num([1,0,1,1,1,0,1,1,1,0,1,0])

In [ ]:
model.predict